<span STYLE="font-size:150%"> 
    Plot drug release parameters
</span>

Docker image: gnasello/datascience-env:2023-01-27 \
Latest update: 20 March 2023

# Install packages

In [ ]:
# Install from CRAN
install.packages("investr", dep = TRUE)

# Load libraries

In [ ]:
library(reshape2)
library(stringr)
library(ggplot2)
library(dplyr)
library(investr)
library(ggpubr)
library(latex2exp)

How to Reuse Functions That You Create In Scripts, [tutorial](https://www.earthdatascience.org/courses/earth-analytics/multispectral-remote-sensing-data/source-function-in-R/)

In [ ]:
source("r_utils/ggplot_utils.R")
source("r_utils/stats_utils.R")

In [ ]:
filename <-  "2023-03-28_GN011_data.csv"
drugname <- 'AS2863619'

In [ ]:
# Import the data and look at the first six rows
df <- read.csv(file = filename)
head(df)

## Drop samples 4s

These samples were removed becauase they were smaller than all the others.

[Remove Row by Multiple Condition](https://www.geeksforgeeks.org/how-to-conditionally-remove-rows-in-r-dataframe/)

In [ ]:
df <- subset(df, df$sample != 4)
head(df)

## Drop samples 6-AS

In [ ]:
df <- subset(df, df$group != '6-AS' )
head(df)

## Summarize the data

The function below will be used to calculate the mean and the standard deviation, for the variable of interest, in each group. See [tutorial](http://www.sthda.com/english/wiki/ggplot2-line-plot-quick-start-guide-r-software-and-data-visualization#line-graph-with-error-bars)

In [ ]:
df1 <- data_summary(df, varname="cum_sum", 
                    groupnames=c("group", "day", "Laponite"))
df1

# Plot release kinetics curve

## Line plot

In [ ]:
p <- ggplot(df1, aes(x=day, y=cum_sum, group=group, color=group)) + 
            geom_errorbar(aes(ymin=cum_sum-sd, ymax=cum_sum+sd), width=.5, size=0.8) +
            geom_line(size=1) + geom_point(size=1.25)

img <- ggplotMinAethetics(p, width=5, height=3.25,
                          title=drugname,
                          xlabel='Time (Days)', 
                          ylabel='Cumulative release (%)', 
                          scale_color='npg') + ylim(0, 101)+ 
        scale_color_discrete(labels=c('- Laponite', '+ Laponite'), name = '')
img

## Regression model

In [ ]:
p <- ggplot(df, aes(x = day, y = cum_sum, color=group, fill = group)) + 
     geom_point(size = 3, alpha=0.7)+
     geom_smooth(method = lm, formula = y ~ log(x+1), alpha=0.1)

img <- ggplotMinAethetics(p, width=5, height=3.25,
                          title=drugname,
                          xlabel='Time (Days)', 
                          ylabel='Cumulative release (%)', 
                          scale_color='npg'
                         ) + ylim(0, 101)+ 
        scale_color_discrete(labels=c('- Laponite', '+ Laponite'), name = '')+
        scale_fill_discrete(labels=c('- Laponite', '+ Laponite'), name = '')
img

# Get drug release parameters

## Logarithmic regression

Linear Regression and group by in R, [stackoverflow](https://stackoverflow.com/questions/1169539/linear-regression-and-group-by-in-r)

[`plotFit`](https://www.rdocumentation.org/packages/investr/versions/1.4.2/topics/plotFit): Plotting fitted models

In [ ]:
group_regress <- function(data){
       
    model = lm(cum_sum~log(day+1), data=data)
    
    plotFit(model, data = data,
            lwd.fit = 2, cex = 1.2, pch = 21, bg = "lightskyblue", 
            lwd = 2, xlab = "Log dose", ylab = "Probability",
            ylim=c(0,105), xlim=c(0,5)
           )
    
    print(data$group[1])
    
    model
    
    }

#fit log model
fitted_models <- df %>% 
                    group_by(group) %>% 
                        do( model = group_regress(data=.))

## Burst release

It is determined by the drug release fraction at 24 hours

In [ ]:
k_df <- data.frame(matrix(ncol = 3, nrow = 0))

for(i in 1:nrow(fitted_models)) {       # for-loop over rows
    
    k <- predict(fitted_models$model[[i]], data.frame(day = 1), interval = "confidence")
    
    k_df <- rbind(k_df, k)

}

rownames(k_df) <- NULL
k_df['group'] <- fitted_models['group']

k_df

Plot burst release parameter

In [ ]:
p <- ggbarplot(k_df, x = "group", y = "fit", fill = "group") + 
            geom_errorbar(aes(ymin=lwr, ymax=upr), width=.2, size=0.5)

ggplotMinAethetics(p, width=1.6, height=3.8,
                   title=drugname,
                   plot.title=element_text(size = 13),
                   xlabel=' ', 
                   ylabel='Burst release (k, %)', 
                   scale_color='npg',
                   legend.position="none"
                  ) + theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))+ 
scale_x_discrete(labels=c("4-AS" = "- Laponite", "4-AS-L" = "+ Laponite"))

## Half-life (t1/2)

It is determined by the time when the fraction of released drug reaches 50%

In [ ]:
cond <- fitted_models$group[2]

invest(fitted_models$model[[2]], y0 = 50, x0.name='time', 
       mean.response = TRUE,
       lower=0.01, upper=20, # lower and upper are estimates based on your data 
       data=df[df$group==cond,]) 

In [ ]:
cond <- fitted_models$group[1]

invest(fitted_models$model[[1]], y0 = 50, x0.name='time',
       mean.response = TRUE,
       lower=0, upper=1000, # lower and upper are estimates based on your data 
       data=df[df$group==cond,]) 

In [ ]:
thalf_df <- data.frame(matrix(ncol = 3, nrow = 0))

for(i in 1:nrow(fitted_models)) {       # for-loop over rows
    
    t <- predict(fitted_models$model[[i]], data.frame(day = 1), interval = "confidence")
    cond <- fitted_models$group[i]
       
    t <- invest(fitted_models$model[[i]], y0 = 50, x0.name='time', 
                mean.response = TRUE,
                lower=0.1, upper=1000, # lower and upper are estimates based on your data 
                data=df[df$group==cond,]) 
    
    thalf_df <- rbind(thalf_df, t)

}

rownames(thalf_df) <- NULL
thalf_df['group'] <- fitted_models['group']

thalf_df <- subset(thalf_df, select = -c(interval) )
thalf_df

In [ ]:
p <- ggbarplot(thalf_df, x = "group", y = "estimate", fill = "group") + 
               geom_errorbar(aes(ymin=lower, ymax=upper), width=.2, size=0.5)

ggplotMinAethetics(p, width=1.6, height=3.8,
                   title='AS2863619',
                   plot.title=element_text(size = 13),
                   xlabel=' ', 
                   ylabel=TeX('Half-life (t$_{1/2}$, days)'), 
                   scale_color='npg',
                   legend.position="none"
                  ) + theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))+ 
scale_x_discrete(labels=c("4-AS" = "- Laponite", "4-AS-L" = "+ Laponite"))

## Save release parameters

In [ ]:
colnames(k_df) <- colnames(thalf_df)

In [ ]:
k_df

In [ ]:
write.csv(k_df, "burst_release_laponite.csv", row.names=FALSE)

In [ ]:
thalf_df

In [ ]:
write.csv(thalf_df, "half_life_laponite.csv", row.names=FALSE)